# Fine-tunning BYOL on Parihaka Dataset

## Imports

In [1]:
from common import get_data_module, get_trainer_pipeline
import torch
from minerva.models.ssl.byol import (
    BYOL
)
from functools import partial
import os

## Variaveis

In [2]:
# root_data_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/images"
# root_annotation_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/annotations"

root_data_dir = "../../../../../../../shared_data/seam_ai_datasets/seam_ai/images"
root_annotation_dir = "../../../../../../../shared_data/seam_ai_datasets/seam_ai/annotations"

print(os.path.exists(root_data_dir))
print(os.path.exists(root_annotation_dir))



img_size = (1008, 784)          # Change this to the size of the images in the dataset
model_name = "byol"             # Model name (just identifier)
dataset_name = "seam_ai"        # Dataset name (just identifier)
single_channel = False          # If True, the model will be trained with single channel images (instead of 3 channels)

log_dir = "./logs"              # Directory to save logs
batch_size = 8                  # Batch size    
seed = 42                       # Seed for reproducibility
num_epochs = 75                 # Number of epochs to train
is_debug = True                 # If True, only 3 batch will be processed for 3 epochs
accelerator = "gpu"             # CPU or GPU
devices = 1                     # Num GPUs

True
True


## Data Module

In [3]:
data_module = get_data_module(
    root_data_dir=root_data_dir,
    root_annotation_dir=root_annotation_dir,
    img_size=img_size,
    batch_size=batch_size,
    seed=seed,
    single_channel=single_channel
)

data_module

DataModule
    Data: ../../../../../../../shared_data/seam_ai_datasets/seam_ai/images
    Annotations: ../../../../../../../shared_data/seam_ai_datasets/seam_ai/annotations
    Batch size: 8

In [4]:
# Just to check if the data module is working
data_module.setup("fit")
train_batch_x, train_batch_y = next(iter(data_module.train_dataloader()))
train_batch_x.shape, train_batch_y.shape

(torch.Size([8, 3, 1008, 784]), torch.Size([8, 1, 1008, 784]))

## **** Create and Load model HERE ****

In [ ]:
import torchvision.models.segmentation as models
from minerva.models.nets.image.deeplabv3 import DeepLabV3Backbone
import os 
from torch import nn

# wheights_path = "/workspaces/HIAAC-KR-Dev-Container/shared_data/notebooks_e_pesos/backbones_byol/V1/V1_E300_B32_S256_both_N.pth"
wheights_path = "../../../../../../../shared_data/notebooks_e_pesos/backbones_byol/V1/V1_E300_B32_S256_seam_ai.pth"
print(os.path.exists(wheights_path))

backbone = models.deeplabv3_resnet50().backbone
backbone.load_state_dict(torch.load(wheights_path))

True


<All keys matched successfully>

In [8]:
from minerva.models.nets.image.deeplabv3 import DeepLabV3, DeepLabV3PredictionHead
from torch import Tensor
from collections import OrderedDict


class DeepLabV3_2(DeepLabV3):
    
    def forward(self, x: Tensor) -> Tensor:
        x = x.float()
        input_shape = x.shape[-2:]
        h = self.backbone(x)
        if isinstance(h, OrderedDict):
            h = h['out']
        z = self.fc(h)
        # Upscaling
        return nn.functional.interpolate(
            z, size=input_shape, mode="bilinear", align_corners=False
        )

In [10]:
from minerva.models.nets.image.deeplabv3 import DeepLabV3, DeepLabV3PredictionHead

pred_head = DeepLabV3PredictionHead(num_classes=6)

model = DeepLabV3_2(
    backbone = backbone,
    pred_head = pred_head,
    loss_fn = torch.nn.CrossEntropyLoss(),
    num_classes = 6,
    learning_rate = 0.001,
)

In [11]:
# ckpt_file = "/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/f3/dinov2_vits14_pretrain.pth"
# ckpt = torch.load(ckpt_file, map_location="cpu")


# Only using the backbone with random prediction head. No need for this step

## Pipeline

In [12]:
pipeline = get_trainer_pipeline(
    model=model,
    model_name=model_name,
    dataset_name=dataset_name,
    log_dir=log_dir,
    num_epochs=num_epochs,
    accelerator=accelerator,
    devices=devices,
    is_debug=is_debug,
    seed=seed,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42


Log directory set to: /workspaces/workspace/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/byol/seam_ai


In [13]:
pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/fabric/loggers/csv_logs.py:269: Experiment logs directory ./logs/byol/seam_ai exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory ./logs/byol/seam_ai/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Pipeline info saved at: /workspaces/workspace/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/byol/seam_ai/run_2024-12-16-18-06-024023ac1417fc4d23b385ba397a6f5b8c.yaml



  | Name     | Type                    | Params | Mode 
-------------------------------------------------------------
0 | backbone | IntermediateLayerGetter | 23.5 M | train
1 | fc       | DeepLabV3PredictionHead | 16.1 M | train
2 | loss_fn  | CrossEntropyLoss        | 0      | train
-------------------------------------------------------------
39.6 M    Trainable params
0         Non-trainable params
39.6 M    Total params
158.540   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 3/3 [00:04<00:00,  0.62it/s, v_num=m_ai, val_loss=33.70, train_loss=1.430]Pipeline info saved at: /workspaces/workspace/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/byol/seam_ai/run_2024-12-16-18-06-024023ac1417fc4d23b385ba397a6f5b8c.yaml


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
print(f"Checkpoint saved at {pipeline.trainer.checkpoint_callback.last_model_path}")

Checkpoint saved at ./logs/byol/seam_ai/checkpoints/last-v1.ckpt
